# Evaluate the existing topics
Review misspelled words
Calculate Coherance Score

In [140]:
import sys
import os

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path+"/scripts")
    sys.path.append(module_path+"/classes")

review_corpus_path = "../data/tech_review_sent_corpus.pkl"
vocab_path = "../data/glove-tech-revew-vocab.txt"
emb_filename = '../models/w2v_embedding'
aspect_file_path = "../results/aspect.json"
aspect_model_path = "../results/model_param"
vocab_path = "../data/vocab-text-review.txt"



In [65]:
import json

with open("../results/aspect.json") as f:
    aspects = json.load(f)
    
print(aspects.keys())

dict_keys(['Aspect 0:', 'Aspect 1:', 'Aspect 2:', 'Aspect 3:', 'Aspect 4:', 'Aspect 5:', 'Aspect 6:', 'Aspect 7:', 'Aspect 8:', 'Aspect 9:', 'Aspect 10:', 'Aspect 11:', 'Aspect 12:', 'Aspect 13:', 'Aspect 14:'])


Get topics words

In [66]:
vocab = []
for aspect, words in aspects.items():
    for word, score in words.items():
        vocab.append(word)

vocab = list(set(vocab))
vocab.sort()


In [67]:
from nltk.metrics.distance import edit_distance, jaro_winkler_similarity

data = {}
for i, word in enumerate(vocab):
    for j, alt_word in enumerate(vocab):
        score = jaro_winkler_similarity(word, alt_word)
        if (score >= .9) & (score < 1):
            try:
                data[word][alt_word] = score
            except:
                data[word] = {}
                data[word][alt_word] = score


In [68]:
for key, alt in data.items():
    for word, score in alt.items():
        print('"'+key+'" : "'+word+'", #'+str(score))
    

"10a" : "10am", #0.9416666666666667
"10am" : "10a", #0.9416666666666667
"10hr" : "10hrs", #0.96
"10hr" : "1hr", #0.9249999999999999
"10hrs" : "10hr", #0.96
"10pm" : "1pm", #0.9249999999999999
"11pm" : "1pm", #0.9249999999999999
"12hrs" : "2hrs", #0.9333333333333332
"12pm" : "1pm", #0.9249999999999999
"12pm" : "2pm", #0.9166666666666666
"15th" : "5th", #0.9166666666666666
"18th" : "8th", #0.9166666666666666
"1hr" : "10hr", #0.9249999999999999
"1pm" : "10pm", #0.9249999999999999
"1pm" : "11pm", #0.9249999999999999
"1pm" : "12pm", #0.9249999999999999
"25th" : "5th", #0.9166666666666666
"2hrs" : "12hrs", #0.9333333333333332
"2pm" : "12pm", #0.9166666666666666
"30am" : "3am", #0.9249999999999999
"30am" : "730am", #0.9333333333333332
"30min" : "30mins", #0.9666666666666667
"30mins" : "30min", #0.9666666666666667
"30pm" : "3pm", #0.9249999999999999
"3am" : "30am", #0.9249999999999999
"3pm" : "30pm", #0.9249999999999999
"3years" : "3yrs", #0.9111111111111111
"3yrs" : "3years", #0.9111111111111

# Create coherance score

In [90]:
import pandas as pd
import pickle

review_corpus_path = "../data/tech_review_sent_corpus.pkl"

with open(review_corpus_path,"rb") as f:
    tech_review_corpus = pickle.load(f)

print(len(tech_review_corpus))
reviews = pd.DataFrame(tech_review_corpus).review.tolist()

156991


In [145]:
import ABAE.reader as dataset

vocab, train_x, overall_maxlen = dataset.get_data(reviews, vocab_path, vocab_size=0, maxlen=115)

vocab_inv = {}
for w, ind in vocab.items():
    vocab_inv[ind] = w

 Creating vocab ...
   944204 total words, 22788 unique words
 Reading dataset ...
  train set
   <num> hit rate: 1.24%, <unk> hit rate: 0.00%


In [152]:
processed_docs = []
for review in train_x:
    processed_docs.append([vocab_inv[i] for i in review ])

processed_docs[:2]

[['great', 'people', 'care', 'make', 'difference'],
 ['management',
  'care',
  'employee',
  'well',
  'providing',
  'continued',
  'learn',
  'opportunity']]

In [107]:
import json

with open("../results/aspect.json") as f:
    aspects = json.load(f)
    
topics = []
for aspect, words in aspects.items():
    topic_words = []
    for word, score in words.items():
        topic_words.append(word)
    topics.append(topic_words)
    

In [153]:
from datetime import datetime
from gensim.corpora.dictionary import Dictionary

start = datetime.now()

dictionary = Dictionary(processed_docs)
dictionary.add_documents([["<num>","<unk>"]])

corpus = [dictionary.doc2bow(text) for text in processed_docs]
end = datetime.now()
print("Time taken: %s" % (end - start))

Time taken: 0:00:02.394423


In [159]:
from gensim.models import CoherenceModel
import numpy as np

start = datetime.now()
c_v = []
flags = []
for n, topic in enumerate(topics):
    cm = CoherenceModel(topics=[topic], texts = processed_docs, corpus=corpus, dictionary=dictionary, coherence='c_v')
    c_v.append(cm.get_coherence())
end = datetime.now()
print("Time taken: %s" % (end - start))

np.mean(c_v)


Time taken: 0:00:08.034366


0.7105521707661687

In [221]:
# https://github.com/derekgreene/topic-model-tutorial/blob/master/3%20-%20Parameter%20Selection%20for%20NMF.ipynb
import gensim
from itertools import combinations
from scipy.spatial.distance import cosine

w2v_model = gensim.models.Word2Vec.load(emb_filename)

def getVector(w2v_model, word):
    try:
        v = w2v_model.wv[word]
    except:
        v = np.repeat(.00000001, w2v_model.vector_size)
    return v

def calculate_coherence( w2v_model, term_rankings ):
    overall_coherence = 0.0
    for topic_index in range(len(term_rankings)):
        # check each pair of terms
        pair_scores = []
        for pair in combinations( term_rankings[topic_index], 2 ):
            pair_scores.append( 1- cosine(getVector(w2v_model, pair[0]), getVector(w2v_model,pair[1])) )
        # get the mean for all pairs in this topic
        topic_score = sum(pair_scores) / len(pair_scores)
        overall_coherence += topic_score
    # get the mean score across all topics
    return overall_coherence / len(term_rankings)

calculate_coherence(w2v_model, topics)

(0.2904856673678078, 0.19806547701218558)

In [220]:
topic_score

NameError: name 'topic_score' is not defined